In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [34]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [35]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))
x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
y = train[:,0]
y = np.reshape(y, (-1, 1))

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 1)
(2048, 1)


In [36]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [37]:
x_total = x.copy()

def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6144, 28, 28, 1)


In [40]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(y_total.shape)

(6144, 10)


In [39]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)

en = OneHotEncoder()
x_letter_total = en.fit_transform(x_letter_total).toarray()
    
print(x_letter_total.shape)

(6144, 26)


In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True, stratify=y_total)

print(x1_train.shape)
print(x1_val.shape)
print(y1_train.shape)
print(y1_val.shape)

x1_letter_train = x_letter_total[:x1_train.shape[0],:]
x1_letter_val = x_letter_total[x1_train.shape[0]:,:]
print(x1_letter_train.shape)
print(x1_letter_val.shape)

input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu')(x2)
outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)

model.summary()

model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

cp = ModelCheckpoint('./models/{epoch:02d}-{val_accuracy:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit([x1_train, x1_letter_train], y1_train,
                    validation_data=([x1_val, x1_letter_val], y1_val), 
                    batch_size=64, epochs=50, verbose=1, callbacks = [cp])

x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255

x_letter_test = test['letter'].values
x_letter_test = x_letter_test[:, np.newaxis]
en = OneHotEncoder()
x_letter_test = en.fit_transform(x_letter_test).toarray()

submission = pd.read_csv('data/submission.csv')
submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)
submission.to_csv('data/submission(CNN).csv', index=False)